In [1]:
from IPython.display import Image, display
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from time import sleep
from PIL import Image as Imag
import pyautogui
import pandas as pd
from copy import deepcopy

In [2]:
# Legal moves list
legal=['blue', 'red', 'green', 'violett']

In [3]:
def closeButtonLookup():
    '''Looks for close button and presses it returns False if close button is not found, else - True'''
    if bool(pyautogui.locateCenterOnScreen('Images/btn_close.png', confidence=0.7))==True:
        x, y = pyautogui.locateCenterOnScreen('Images/btn_close.png', confidence=0.7)
        pyautogui.click(x/2, y/2, duration=0.5, button='right')
        print('OK,I am out of here!')
        return True
    return False

In [45]:
class GameState():
    '''Game state, board = dataframe, 'frm' and 'at' = lists of row/colum values of origin and current position'''
    def __init__(self,  board, parent=None, frm=None, at=None):
        self.board = board
        self.parent = parent
        self.frm = frm
        self.at = at
        self.matches = list()
        self.children = list()

    def find_matches(self):
        
        for child in self.children:
    #         TODO: loop over legal candidates
            res_vert = {i : select_type('red', self.children[1].board, col=i) for i in self.children[1].board.columns 
                if select_type('red', self.children[1].board, col=i).any()}
            for key, val in res_vert.items():
                match = list(find_match(val))
                if match:
                    print(self.board.loc[match,key])
                    self.matches.append( Match(self.board.loc[match,key], val.all()))

#             print(find_match(val))
#             print(val.all())

In [30]:
class Match():
    '''Match contains info on size, position and type of candy'''
    def __init__(self, typ, pos):
        self.size = pos.shape[0]
        self.pos = pos
        self.typ = typ       

In [6]:
def find_match(selected): 
    '''Takes Series as an input, returns index of a match3 in the series if there is, else: False'''
    match=[i for i in selected.index]
    diff = [1]+[i-j for j,i in zip(match[:-1], match[1:])]
    diff = np.absolute(np.array(diff)) 
    mask = diff == 1
    if selected[mask].size>=3:
        return selected[mask].index
    return False

## Create board

#### Get input for board height and width.

In [7]:
# board_w = int(input('Borad width is:  '))
board_w = 5

In [8]:
# board_h = int(input('Borad width is: '))
board_h = 5

In [9]:
board = pd.DataFrame('0', index=range(board_h), columns=range(board_w))
Root = GameState(board) 

In [78]:
Root.board

,0,1,2,3,4
0,0,0,0,0,0
1,0,red,red,0,red
2,0,blue,blue,blue,red
3,0,blue,0,0,red
4,0,0,0,0,0


#### Add some candy manually for testing purpouse

In [11]:
Root.board.loc[1,[1,2,4]]='red'
Root.board.loc[2,[1,2,3]]='blue'
Root.board.loc[[1,2,3],4]='red'
Root.board.loc[3,1]='blue'

## Look for the matches

In [12]:
def select_type(typ, board, ind=board.index, col=board.columns):
    '''Select all instances of a given type in a Series = board.iloc[ind, col] where board is Pandas DataFrame
    one of the kwargs MUST be given'''
    selected = board.loc[ind,col][board.loc[ind,col]==typ]
#     print(list(filter(lambda x: type(x)==int, [ind,col])))
    if selected.size>=3:
        return selected
    return pd.Series(dtype='object')

In [13]:
def move_to(board, fr, to):
    '''Takes a Dataframe, and two lists of row/col values, swaps values in specified cells. 
        Outputs resulting Data frame '''
    board = board.copy()
    board.loc[fr[0], fr[1]], board.loc[to[0], to[1]] = board.loc[to[0], to[1]], board.loc[fr[0], fr[1]]
    return board

In [65]:
def generate_children(board, pos):
#     TODO: make pos a positional arg.
    '''Generate valid moves from a given tile, pos=list of row/col values'''
    positions = np.array(pos)
    directions = np.array([[1, 0], [0, 1], [-1, 0], [0, -1]])
    moves = np.add(positions, directions)
    # Filter the values outside of the board
    moves = moves[(0<=moves[:,0]) & (moves[:,0]<board_w)]
    moves = moves[(0<=moves[:,1]) & (moves[:,1]<board_h)]
    # filter the legal moves and save them in a children nodes attribute
    children=[GameState(move_to(board.copy(), pos, i), None, frm=pos, at=i) for i in moves if 
              board.iloc[i[0],i[1]] in legal]
    return children

In [101]:
from itertools import product, chain

In [96]:
all_tiles = list(product(range(5),range(5)))

In [97]:
candidates = [generate_children(Root.board, tile) 
              for tile in all_tiles if Root.board.iloc[tile[0], tile[1]] in legal ]

In [99]:
print(f'This is a result of a move from {candidates[1][0].frm} to {candidates[1][0].at}')

This is a result of a move from (1, 2) to [2 2]


In [105]:
next(chain(*candidates))

### Look for matches in one of resulting boards.

Column-wise.

In [108]:
candidate = next(chain(*candidates))

In [110]:
typ = candidate.board.iloc[candidate.frm[0], candidate.frm[1]]

'blue'

In [24]:
res_vert = {i : select_type(typ, candidate.board, col=i) for i in candidate.board.columns 
            if select_type(typ, candidate.board, col=i).any()}

And row-wise

In [25]:
res_horyz = {i : select_type(typ, candidate.board, ind=i) for i in candidate.board.index 
             if select_type(typ, candidate.board, ind=i).any()}

In [27]:
for key, val in res_horyz.items():
    print(list(find_match(val)))
    print(val.all())
key    

[1, 2, 3]
blue


2

In [39]:
m=Match('blue',Root.board.iloc[2, [1, 2, 3]])
Root.matches.append(m)
Root.matches[0].typ

'blue'

In [47]:
Root.matches[0].pos

1    blue
2    blue
3    blue
Name: 2, dtype: object

In [32]:
children[1].find_matches()

In [46]:
children[1].matches

[]

Find where it is and cahnge output into *nx2* array

In [ ]:
# screen size
pyautogui.size().height

In [ ]:
tile_w = 500/7
tile_h = 524/8

In [ ]:
offset_x = tile_w/2
offset_y = tile_h/2